### 🔬PARQUET LAB

In [2]:
import warnings
import os
from os import listdir,getenv
import zipfile
import glob
import src.utils as utils
from src.utils import handle_count_files,handle_size_files,handle_unzip_files,handle_new_dir_parquet,handle_csv_to_parquet,handle_clean_tmp

from datetime import datetime
import time


# diretórios existentes
path_root = getenv("PATH_ROOT")
bucket_parquet = getenv("BUCKET_PARQUET")
path_tmp = f"{path_root}tmp/"

# variáveis
target = 'geoleads'
final_target = f'{path_root}{target}/'
target_parquet = f"{bucket_parquet}/{target}/"

In [6]:
'''
ETAPA 1
→ Extrai os arquivos da pasta target e move para pasta temporária
→ Calcula o volume e tamanho total de arquivos na pasta que será convertida (temp)
'''
inicio = datetime.now()
print(f"🚩 START >> {inicio.strftime('%H:%M:%S')} <<")
print("STEP 1 ----------------•")
print(f"ORIGINAL PATH: {final_target} | TEMP PATH: {path_tmp} ")
handle_unzip_files(final_target,path_tmp)
qtd_1 = handle_count_files(path_tmp)
size_1 = handle_size_files(path_tmp)
print(f"FILES: {qtd_1}")
print(f"TOTAL SIZE: {size_1/ (1024 * 1024):.2f} MB")
print("___________________________________")

'''
ETAPA 2
→ Cria novo diretório com o nome target no bucket de parquet's
→ Converte todos os arquivos na pasta temp para parquet
→ Move para nova pasta
'''
print("STEP 2 ----------------•")
handle_new_dir_parquet(bucket_parquet,target)
print(f"PARQUET PATH: {target_parquet}")
handle_csv_to_parquet(path_tmp,target_parquet)
qtd_2 = handle_count_files(target_parquet)
size_2 = handle_size_files(target_parquet)
print(f"PARQUET FILES: {qtd_2}")
print(f"TOTAL SIZE: {size_2/ (1024 * 1024):.2f} MB")
print("___________________________________")

'''
ETAPA 3
→ Calcula economia de espaço após conversão
'''
print("ℹ INFOS ----------------•")
# OPEN IA
# Calcula a diferença absoluta e a porcentagem de economia
diferenca = size_1 - size_2
if size_1 > 0:  # Evita divisão por zero
    economia_percentual = (diferenca / size_1) * 100
else:
    economia_percentual = 0
print(f"SIZE SAVED: {diferenca/ (1024 * 1024):.2f} MB")
print(f"% REDUCTION: {economia_percentual:.2f}%")
print("___________________________________")
print(f"🚩 START >> {inicio.strftime('%H:%M:%S')} <<")
fim = datetime.now()
print(f"🏁 END >> {fim.strftime('%H:%M:%S')} <<")
# Calcula o tempo total
tempo_total = fim - inicio
print(f"⏳: {tempo_total}")
handle_clean_tmp(path_tmp)

🚩 START >> 15:33:30 <<
STEP 1 ----------------•
ORIGINAL PATH: C:/Dados/SRCS/csv_in/geoleads/ | TEMP PATH: C:/Dados/SRCS/csv_in/tmp/ 
FILES: 13
TOTAL SIZE: 1243.60 MB
___________________________________
STEP 2 ----------------•
PARQUET PATH: C:/Dados/SRCS/bucket/parquet/geoleads/
PARQUET FILES: 13
TOTAL SIZE: 91.02 MB
___________________________________
ℹ INFOS ----------------•
SIZE SAVED: 1152.58 MB
% REDUCTION: 92.68%
___________________________________
🚩 START >> 15:33:30 <<
🏁 END >> 15:34:08 <<
⏳: 0:00:38.479311


In [7]:

# Init Notebook
import src.utils as utils                  # configs and functions
import src.models.marts as mql             # marts options
import src.models.sources as soql          # sources options
import src.models.staging.query as qsql    # staging query
import src.models.staging.update as update # staging update
import src.models.staging.create as create # staging create
import duckdb
import pandas as pd
import warnings
                        
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

con = duckdb.connect(utils.DUCKDB_DATABASE) # type: ignore

In [29]:
'''
q=f"""
drop table if exists tbl_parquet_lab;
create table tbl_parquet_lab (target varchar(255), files int,tt_size_csv varchar(255),tt_size_parquet varchar(255), tt_size_saved varchar(255),tt_reduction varchar(255))
"""
con.execute(q)
'''

In [8]:

q=f"""
insert into tbl_parquet_lab
(target, files, tt_size_csv,tt_size_parquet, tt_size_saved,tt_reduction)
values
('{target}',{qtd_1},'{f"{size_1/ (1024 * 1024):.2f} MB"}','{f"{size_2/ (1024 * 1024):.2f} MB"}','{f"{diferenca/ (1024 * 1024):.2f} MB"}','{f"{economia_percentual:.2f}%"}');
select * from tbl_parquet_lab
"""
con.execute(q).fetch_df()

,target,files,tt_size_csv,tt_size_parquet,tt_size_saved,tt_reduction
0,cadastro_correto_completo,12,2514.25 MB,100.62 MB,2413.63 MB,96.00%
1,analitico_quebra,12,6914.07 MB,578.98 MB,6335.09 MB,91.63%
2,d1_backlog_comercial,11,1403.01 MB,100.70 MB,1302.31 MB,92.82%
3,desistencia_recadastro,11,1370.68 MB,106.04 MB,1264.63 MB,92.26%
4,geoleads,13,1243.60 MB,91.02 MB,1152.58 MB,92.68%


In [1]:
q=f"""
select * from select * from tbl_parquet_lab
where 1=1

"""
con.execute(q).fetch_df().T

NameError: name 'con' is not defined